# Project 1: Introduction to Airfoils and XFOIL

#### Author: Jack Comey

##### Student ID: 915323775
##### Due Date: 2:00 PM Monday 26 OCT 2020

###### Code Imports and Basic Function Definition:

## 1 | Airfoil Geometry Characteristics

This problem is a plotting excersize, and consists of plotting two NACA airfoils given 

### 1.1 | 

### 1.2 | 

## 2 | 

## 3 | 